Llegir arxiu

In [ ]:
import numpy as np

# Función para leer el archivo y extraer las matrices
def leer_archivo(nombre_archivo):
    c = []
    A = []
    b = []

    with open(nombre_archivo, 'r') as file:
        for line in file:
            if line.startswith('c='):
                c = [int(x) for x in line.split()[1:]]
            elif line.startswith('A='):
                while True:
                    line = next(file)
                    if line.startswith(' '):
                        A.append([int(x) for x in line.split()])
                    else:
                        break
            elif line.startswith('b='):
                b = [int(x) for x in line.split()[1:]]
    
    # Convertir las listas a arrays numpy
    c = np.array(c)
    A = np.array(A)
    b = np.array(b)
    
    return c, A, b

# Nombre del archivo
nombre_archivo = 'informacion.txt'

# Llamar a la función para leer el archivo
c, A, b = leer_archivo(nombre_archivo)

# Mostrar las matrices
print("Matriz c:")
print(c)
print("\nMatriz A:")
print(A)
print("\nVector b:")
print(b)


Classe

In [ ]:
class SIMPLEX:
    def __init__(self,Coeficientes, Arestricciones, bsolucion):
    
    def encontrar_SBF_fase1:
    
    def optima_variablesentrada:
    
    def DBF_:
    
    def longitudpaso_quiensale:
    
    def actualizar y repetir: